In [ ]:
import requests #common libraries to install for any type of scraping
import pandas as pd
import re
from bs4 import BeautifulSoup
from random import randint,choice
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])

# Fintech Data Scraping

In [ ]:
r = requests.get('https://courses.cfte.education/ranking-of-largest-fintech-companies/',verify=False)

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

In [ ]:
a = soup.find_all("td")
i = 0
for i in range(1,len(a),i+11):
    if str(a[i].string) != 'None' and str(a[i+1].string) == 'United States':
        print(str(a[i].string))

# Clutch Scraping

In [ ]:
headers = [{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'},
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33'},
{'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'},
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 OPR/90.0.4480.84'}]

data = {
  "name": [],
  "description": [],
  'Employees': [],
  'Founded': [],
  "all_locations": [],
  "Service lines": [],
  "Client focus":[],
  "Industry focus":[],
  "Clients Name": [],
  "Website": [],
  "linkedin": []
}
graph = ['Service lines', 'Client focus', 'Industry focus']
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='search links')
start_page = int(dataframe['start page'][len(dataframe)-1])
last_page = int(dataframe['last page'][len(dataframe)-1])
for i in range(start_page, last_page+1):
    try:
        r = requests.get(dataframe['link'][len(dataframe)-1]+str(i),verify=False,headers = headers[randint(0,3)]) #put link here
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            company_name = soup.find_all(class_="company_info")
            website_profile = soup.find_all(class_="website-profile")
            print('succesfully loaded Page'+str(i+1))
            for j in range(len(company_name)):
                try:
                    r1 = requests.get('https://clutch.co'+website_profile[j].a['href'], verify=False, headers = headers[randint(0,3)])
                    soup1 = BeautifulSoup(r1.content, 'html.parser')
                    data['name'].append(" ".join(company_name[j].a.string.split()))
                    if r1.status_code == 200:
                        graph_name = []
                        focus = soup1.find_all(class_='chartAreaContainer spm-bar-chart')
                        if soup1.find(class_='field-name-clients') != None:
                            data['Clients Name'].append(soup1.find(class_='field-name-clients').find('div',class_='field-item').text.strip())
                        else:
                            data['Clients Name'].append(' ')
                        for d in soup1.find_all(class_='graph-title'):
                            graph_name.append(d.text)
                        for p in graph:
                            if p in graph_name:
                                indx = graph_name.index(p)
                                focus_details = ''
                                for c in focus[indx].find_all('div'):
                                    focus_details += c.text + ' ' + c['data-content'][3:-4] + ', ' 
                                data[p].append(focus_details)
                            else:
                                data[p].append(' ')
                        locations = ''
                        for m in range(len(soup1.find_all(class_ = 'locality'))):
                            locations += soup1.find_all(class_='locality')[m].text + ' ' +soup1.find_all(class_='country-name')[m].text+'; '
                        data['all_locations'].append(locations)
                        if soup1.find(class_='field-name-profile-summary') != None:
                            data['description'].append(soup1.find(class_='field-name-profile-summary').text.strip())
                        else:
                            data['description'].append('')
                        if soup1.find('div', attrs={'data-content':'<i>Employees</i>'}) != None:
                            data['Employees'].append(soup1.find('div', attrs={'data-content':'<i>Employees</i>'}).text.strip())
                            data['Founded'].append(soup1.find('div', attrs={'data-content':'<i>Founded</i>'}).text.strip())
                        else:
                            data['Employees'].append('')
                            data['Founded'].append('')
                        if soup1.find(class_='website-link-a') != None:
                            data['Website'].append(soup1.find(class_='website-link-a').a['href'])
                        else:
                            data['Website'].append('')
                        if soup1.find(class_='linkedin') != None:
                            url = soup1.find(class_='linkedin')['href']
                            data['linkedin'].append(url)
                        else:
                            data['linkedin'].append('')
                        print('successfully scraped company '+str(j+1))
                    else:
                        print('Bad Response from website for company '+str(j+1)+' details')
                        for key in data.keys():
                            if key != 'name':
                                data[key].append('')
                    df = pd.DataFrame(data)
                    with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                        df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                        df.to_excel(writer, sheet_name="result")
                    sleep(randint(3,10))
                except:
                    print('error loading company '+str(j+1)+' details')
                    for key in data.keys():
                        if key != 'name':
                            data[key].append('')
        else:
            print('Bad Response from website for page '+str(i+1)+ ' Skipped to next page.')
    except:
        print('error loading page '+str(i+1)+' Skipped to next page.')


# LinkedIn Scraping

In [ ]:
s=Service(r'') #login code
driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get("https://www.linkedin.com/checkpoint/lg/sign-in-another-account")
WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']")))
username = driver.find_element(By.ID,"username")
username.send_keys("")
pword = driver.find_element(By.ID, "password")
pword.send_keys("")
driver.find_element(By.XPATH, "//button[@type='submit']").click()

### Linkedin scraping 1: when company links are given

In [ ]:
def check_link(i, word):
    if word in i:
        indx = i.index(word)
        return i[:indx]
    else:
        return i
profile = {'name': [],
           'Description': [],
           'Founded': [],
           'Industry': [],
           'Company size': [],
           'Headquarters': [],
           'Total Employees': [],
           'Employees by Geo': [],
           'Specialties':[],
           'Website': []
          }
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='linkedin links')
dataframe['linkedin'] = dataframe['linkedin'].fillna('')
info_heading = ['Industry', 'Company size', 'Headquarters', 'Founded', 'Specialties', 'Website' ]
words = ['about','mycompany','?view','?trk','admin']
j=0
for i in dataframe['linkedin']:
    try:
        if len(i) != 0 and '/company/' in i:
            for word in words:
                i = check_link(i,word)
            if i[-1] != '/':
                i = i + '/'
            driver.get(i +'about/')
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "org-grid__content-height-enforcer")))
            soup1 = BeautifulSoup(driver.page_source, 'html.parser')
            profile['name'].append(dataframe['name'][j])
            if soup1.find('p', class_='break-words white-space-pre-wrap mb5 text-body-small t-black--light') != None:
                profile['Description'].append(soup1.find('p', class_='break-words white-space-pre-wrap mb5 text-body-small t-black--light').text.strip())
            else:
                profile['Description'].append(' ')
            real_info_heading = [k.text.strip() for k in soup1.find_all('dt', class_='text-heading-small')]
            real_info_tag = [m for m in soup1.find_all('dt', class_='text-heading-small')]
            for p in info_heading:
                if p in real_info_heading:
                    indx = real_info_heading.index(p)
                    profile[p].append(real_info_tag[indx].find_next('dd').text.strip()) 
                else:
                    profile[p].append(' ')
            try:
                driver.get(i +'people/')
                WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, "artdeco-carousel__item-container")))
                soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                if soup1.find('h2', class_='t-20 t-black t-bold') != None:
                    profile['Total Employees'].append(soup1.find('h2', class_='t-20 t-black t-bold').text.strip())
                else:
                    profile['Total Employees'].append('')
                if soup1.find('div',class_='artdeco-carousel__item-container') != None:
                    container =  soup1.find('div',class_='artdeco-carousel__item-container')
                    container_items = container.find_all('div',class_='org-people-bar-graph-element__percentage-bar-info truncate full-width mt2 mb1 t-14 t-black--light t-normal')
                    people = ''
                    for s in range(len(container_items)):
                        if s<4:
                            people += container_items[s].text.strip() + '; '
                    profile['Employees by Geo'].append(people)
                else:
                    profile['Employees by Geo'].append('')
                sleep(randint(3,10))
            except Exception as e:
                print("error in people section of company"+str(j+1)+" "+str(e))
                profile['Total Employees'].append('')
                profile['Employees by Geo'].append('')
            df = pd.DataFrame(profile)
            with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                df.to_excel(writer, sheet_name="result")
            print('company '+str(j+1)+" scraped and stored")
    except Exception as e:
        print("error on company"+str(j+1)+str(e))
    j += 1
driver.quit()

### Linkedin scraping 2: when a single search link is given

In [ ]:
profile = {'name': [],
           'Description': [],
           'Founded': [],
           'Industry': [],
           'Company size': [],
           'Headquarters': [],
           'Total Employees': [],
           'Where they live': [],
           'Where they studied':[],
           'What they do':[],
           'What they are skilled at':[],
           'What they studied':[],
           'Specialties':[],
           'Website': [],
           'linkedin link':[]
          }
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='search links')
start_page = int(dataframe['start page'][len(dataframe)-1])
last_page = int(dataframe['last page'][len(dataframe)-1])
info_heading = ['Industry', 'Company size', 'Headquarters', 'Founded', 'Specialties', 'Website' ]
info_people = ['Where they live','Where they studied','What they do','What they are skilled at','What they studied']

for page in range(start_page,last_page+1):
    try:
        driver.get(dataframe['link'][len(dataframe)-1]+str(page))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "entity-result__item")))
        sleep(1)
        print('page '+str(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        companies_single_page = soup.find_all(class_='entity-result__item')
        for i in range(len(companies_single_page)):
            try:
                driver.get(companies_single_page[i].find(class_='app-aware-link')['href']+'about/')
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "org-grid__content-height-enforcer")))
                soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                nav_bar = [i.text.strip() for i in soup1.find_all(class_='org-page-navigation__item-anchor')]
                profile['name'].append(companies_single_page[i].find_all(class_='app-aware-link')[1].text.strip())
                profile['linkedin link'].append(companies_single_page[i].find(class_='app-aware-link')['href'])
                if soup1.find('p', class_='break-words white-space-pre-wrap t-black--light mb5 text-body-small') != None:
                    profile['Description'].append(soup1.find('p', class_='break-words white-space-pre-wrap t-black--light mb5 text-body-small').text.strip())
                else:
                    profile['Description'].append(' ')
                real_info_heading = [k.text.strip() for k in soup1.find_all('dt', class_='text-heading-small')]
                real_info_tag = [m for m in soup1.find_all('dt', class_='text-heading-small')]
                for p in info_heading:
                    if p in real_info_heading:
                        indx = real_info_heading.index(p)
                        profile[p].append(real_info_tag[indx].find_next('dd').text.strip()) 
                    else:
                        profile[p].append(' ')
                if 'People' in nav_bar:
                    try:        
                        driver.get(companies_single_page[i].find(class_='app-aware-link')['href'] +'people/')
                        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, "artdeco-carousel__item-container")))
                        q = 0
                        soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                        profile['Total Employees'].append(soup1.find('h2', class_='t-20 t-black t-bold').text.strip())
                        for y in range(3):
                            for z in range(0,2):
                                if q < 5:
                                    soup1 = BeautifulSoup(driver.page_source, 'html.parser')
                                    container = soup1.find_all('div',class_='artdeco-carousel__item-container')
                                    container_items = container[q].find_all('div',class_='org-people-bar-graph-element__percentage-bar-info truncate full-width mt2 mb1 t-14 t-black--light t-normal')
                                    people = ''
                                    for s in range(len(container_items)):
                                        people += container_items[s].text.strip() + '; '
                                    profile[info_people[q]].append(people)
                                    q += 1
                            driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, "//button[@aria-label='Next']"))
                    except Exception as e:
                        for heading in info_people:
                            profile[heading].append('')
                        profile['Total Employees'].append('')
                        if soup1.find('h2', class_='t-20 t-black t-bold') == None:
                            driver.get_screenshot_as_file("screenshot.png")
                        print('company'+str(i+1)+' got error: '+str(e))
                else:
                    for heading in info_people:
                        profile[heading].append('')
                    profile['Total Employees'].append('')
                df = pd.DataFrame(profile)
                with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                    df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                    df.to_excel(writer, sheet_name="result")
                print('company '+str(i+1)+' saved')
                sleep(randint(3,10))
            except Exception as e:
                print("error in second loop of company "+str(i+1)+" "+str(e))
    except Exception as e:
        print("error in first loop on page "+str(page)+str(e))
driver.quit()

### Linkedin scraping 3: when connection link is given

In [ ]:
profile = {'Name': [],
           'Description': [],
           'Place': [],
           'Profile link':[]
          }
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='search links')
start_page = int(dataframe['start page'][len(dataframe)-1])
last_page = int(dataframe['last page'][len(dataframe)-1])
for page in range(start_page,last_page+1):
    try:
        driver.get(dataframe['link'][len(dataframe)-1]+str(page))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "entity-result__item")))
        sleep(1)
        print('page '+str(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        single_page_profile = soup.find_all(class_='entity-result__item')
        for i in range(len(single_page_profile)):
            if single_page_profile[i].find_all(class_='app-aware-link')[1].span != None:
                profile['Name'].append(single_page_profile[i].find_all(class_='app-aware-link')[1].span.span.text.strip())
            else:
                profile['Name'].append(single_page_profile[i].find_all(class_='app-aware-link')[1].text.strip())
            profile['Profile link'].append(single_page_profile[i].find(class_='app-aware-link')['href'])
            if single_page_profile[i].find(class_='entity-result__primary-subtitle') != None:
                profile['Description'].append(single_page_profile[i].find(class_='entity-result__primary-subtitle').text.strip())
            else:
                profile['Description'].append('')
            if single_page_profile[i].find(class_='entity-result__secondary-subtitle') != None:
                profile['Place'].append(single_page_profile[i].find(class_='entity-result__secondary-subtitle').text.strip())
            else:
                profile['Place'].append('')
            df = pd.DataFrame(profile)
            with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                df.to_excel(writer, sheet_name="result")
            print('profile '+str(i+1)+' saved')
        sleep(randint(3,10))
    except Exception as e:
        print("page "+str(page)+" got error: "+str(e))
        
driver.quit()

### Linkedin scraping 4: Request sending

In [ ]:
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='linkedin links')
profile_links = dataframe['linkedin']
status = {
    'profile link':[],
    'status':[]
}
for i in profile_links:
    try:
        status['profile link'].append(i)
        driver.get(i)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "//div[@class='pv-top-card-v2-ctas ']//li-icon[@type='download']")))
        connect = driver.find_elements(By.XPATH,"//div[@class='pv-top-card-v2-ctas ']//li-icon[@type='connect']")
        pending = driver.find_elements(By.XPATH,"//div[@class='pv-top-card-v2-ctas ']//li-icon[@type='clock']")
        if len(connect) != 0:
            driver.execute_script("arguments[0].click();", connect[0])
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Send now']"))).click()
            print("Connection Request Sent Successfully to: "+str(i))
            status['status'].append("Connection Request Sent Successfully")
        elif len(pending) != 0:
            print("Pending request to: "+str(i))
            status['status'].append("Pending request")
        else:
            print("Already a connection: "+str(i))
            status['status'].append("Already a connection")
        sleep(randint(3,10))      
    except Exception as e:
        print("Sending connection Request Failed for: "+str(i)+" .Due to: "+str(e))
        status['status'].append("Error sending request")
    finally:
        df = pd.DataFrame(status)
        with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
            df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
            df.to_excel(writer, sheet_name="result")
driver.quit()

# Axial Scraping

In [ ]:
s=Service(r'')
driver = webdriver.Chrome(service=s, options=chrome_options)
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='search links')
link = dataframe['link'][len(dataframe)-1]
start_page = dataframe['start page'][len(dataframe)-1]
last_page = dataframe['last page'][len(dataframe)-1]
data = {
    'name':[],
    'website':[],
    'overview':[],
    'industries':[],
    'transactions':[],
    'locations':[],
    'team':[]
}
for i in range(start_page,last_page+1):
    try:
        driver.get(link + str(i) + "/")
        soup = BeautifulSoup(driver.page_source,'html.parser')
        profile_links = soup.find_all(class_='teaser1-title')
        print('successfully loaded page '+str(i))
        for j in range(len(profile_links)):
            data['name'].append(profile_links[j].text.strip())
            try:
                driver.get(profile_links[j].a['href'])
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "grid__col-9")))
                for k in driver.find_elements(By.XPATH, "//span[@class='axl-caret--down ng-star-inserted']"):
                    driver.execute_script("arguments[0].click();", k)
                for l in driver.find_elements(By.XPATH, "//i[@class='material-icons industry-accordion__icon']"):
                    driver.execute_script("arguments[0].click();", l)
                soup1 = BeautifulSoup(driver.page_source,'html.parser')
                if len(soup1.find_all(lambda tag: tag.name == 'a' and tag.get('class') == ['u-link'])) != 0:
                    data['website'].append(soup1.find_all(lambda tag: tag.name == 'a' and tag.get('class') == ['u-link'])[0]['href'])
                else:
                    data['website'].append('')
                if soup1.find('account-profile-overview') != None:
                    data['overview'].append(soup1.find('account-profile-overview').find(class_='grid').text.replace('\n',''))
                else:
                    data['overview'].append('')
                industries = ''
                industry_container = soup1.find_all('div', class_='accordion__tab ng-star-inserted')
                for m in industry_container:
                    industries += m.find(class_='u-text-body').text.replace('\n','') + '\n'
                    for n in m.find_all(class_='u-text-body ng-star-inserted'):
                        industries += n.text.strip() + '; '
                    industries += '\n\n'
                data['industries'].append(industries)
                if soup1.find(id='transactions') != None:
                    data['transactions'].append(" ".join(soup1.find(id='transactions').find(class_='card__title u-text-heading u-text-heading--xxsmall').text.split()))
                else:
                    data['transactions'].append('')
                location_container = soup1.find_all(class_='account-profile-office grid__col-4 grid__col--top-bottom-gutters ng-star-inserted')
                locations = ''
                for o in location_container:
                    locations += ' '.join(o.text.split()[1:])
                    locations += '\n'
                data['locations'].append(locations)
                team = ''
                for p in soup1.find_all(class_='member grid'):
                    team += ' '.join(p.find(class_='grid__col-8').text.split()) + '\n'
                data['team'].append(team)
                print('successfully scraped company '+str(j+1))
                sleep(randint(3,10))
            except:
                print('error loading company '+str(j+1)+' details')
                for key in data.keys():
                    if key != 'name':
                        data[key].append('')
    except:
        print('error loading page '+str(i)+' Skipped to next page.')
df = pd.DataFrame(data)
with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="result") 
driver.quit()

## Life sciences USA scraping

In [ ]:
profile = {'Person': [],
           'Position': [],
           'Organisation': [],
           'Country': [],
           'Org Description': [],
           'Website': [],
           'Linkedin': [],
           'Group': [],
           'Industry': []
          }
s=Service(r'')
driver = webdriver.Chrome(service=s, options=chrome_options)
columns = ['Group', 'Industry']
extra_details = ['Org Description','Website','Linkedin','Group','Industry']
for i in range(84,119):
    try:
        driver.get("https://www.life-sciences-usa.com/person/transatlantic-europe-israel-america-canada-persons-people-addresses-contacts-list-biotech-medtech-1001-1063-43-"+str(i)+"-1-asc.html")
        print("successfully loaded page "+str(i+1)+' .Fetching details of each company now......')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        single_page = soup.find_all('tr')
        for j in range(1,len(single_page)):
            single_row = single_page[j].find_all('td')
            profile['Person'].append(single_row[0].text.strip())
            profile['Position'].append(single_row[1].text.strip())
            profile['Organisation'].append(single_row[2].text.strip())
            profile['Country'].append(single_row[3].text.strip())
            try:
                driver.get("https://www.life-sciences-usa.com"+single_row[0].a['href'])
                sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                col_names = [col.find_all('td')[1].text for col in soup.find_all('tbody')[0].find_all('tr')]
                link = ""
                if col_names[0] == "Organisation":
                    link = soup.find_all('tbody')[0].find_all('tr')[0].find_all('td')[2].a['href']
                if len(link) != 0:
                    try:
                        driver.get("https://www.life-sciences-usa.com"+link)
                        sleep(2)
                        soup = BeautifulSoup(driver.page_source, 'html.parser')
                        if len(soup.find_all('p',class_ = "iconlines")) != 0:
                            website=''
                            linkedin = ''
                            for l in soup.find_all('p',class_ = "iconlines"):
                                if len(l.find_all('img')) != 0:
                                    website = l.text
                                    profile['Website'].append(website)
                                elif "LinkedIn" in l.text:
                                    linkedin = l.text[9:].strip()
                                    profile['Linkedin'].append(linkedin)
                            if len(website) == 0:
                                profile['Website'].append('')
                            if len(linkedin) == 0:
                                profile['Linkedin'].append('')
                        else:
                            profile['Website'].append('')
                            profile['Linkedin'].append('')
                        table = soup.find('table')
                        if table != None:
                            col_names2 = [col.find_all('td')[1].text for col in table.find_all('tr')]
                            prev_ele = list(table.previous_siblings)
                            prev_ele = [i for i in prev_ele if i != '\n']
                            if prev_ele[0].name == 'p' and prev_ele[1].name == 'p':
                                profile['Org Description'].append(prev_ele[1].text.strip())
                            else:
                                profile['Org Description'].append('')
                            for item in columns:
                                if item in col_names2:
                                    indx = col_names2.index(item)
                                    profile[item].append(table.find_all('tr')[indx].find_all('td')[2].text.strip())
                                else:
                                    profile[item].append('')
                            print("successfully scraped company "+str(j))
                        else:
                            for key in extra_details:
                                if key != 'Website' and key != 'Linkedin':
                                    profile[key].append('')
                            print('check this')
                            print("successfully scraped company "+str(j))
                    except Exception as e:
                        print("cant get further company "+str(j)+" details because last page not loaded due to "+str(e))
                        for key in extra_details:
                            profile[key].append('')
                else:
                    print("cant get further company "+str(j)+" details because there is no link on the web page")
                    for key in extra_details:
                        profile[key].append('')        
            except Exception as e:
                print("company "+str(j)+" not loaded due to "+str(e))
                for key in extra_details:
                    profile[key].append('')
            df = pd.DataFrame(profile)
            with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                df.to_excel(writer, sheet_name="result")
            print("successfully saved fetched details of company "+str(j))
            print()
            sleep(randint(3,10))
    except Exception as e:
        print("page "+str(i+1)+" not loaded due to "+str(e))
driver.quit()

# Techreviewer scraping

In [ ]:
profile = {
  "name": [],
  "description": [],
  'Company size': [],
  'Founded': [],
  "Client focus":[],
  'Headquarters':[],
  "Other locations": [],
  "Service Lines": [],
  "Domain focus":[],
  "Frameworks":[],
  "CMS solutions":[],
  "Major Clients": [],
  "Website": [],
  "linkedin": []
}
extra = ['Domain focus', 'CMS solutions', 'Frameworks', 'Service Lines']
s=Service(r'')
driver = webdriver.Chrome(service=s, options=chrome_options)
dataframe = pd.read_excel('MnA Details.xlsx', sheet_name='search links')
start_page = int(dataframe['start page'][len(dataframe)-1])
last_page = int(dataframe['last page'][len(dataframe)-1])
for page in range(start_page,last_page+1):
    try:
        count = 1
        if count != 1:
            driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, "//a[@rel='next']"))
        else:
            driver.get(dataframe['link'][len(dataframe)-1]+str(page))
        print('page '+str(page))
        for i in range(len(driver.find_elements(By.CLASS_NAME, "company-item__profile-btn"))):
            try:
                driver.execute_script("arguments[0].click();", driver.find_elements(By.CLASS_NAME, "company-item__profile-btn")[i])
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                profile['name'].append(soup.find(class_="company-info-header__name").text.strip())
                profile['Website'].append(soup.find(class_='company-info-header__aside').a['href'])
                profile['Company size'].append(soup.find_all(class_='company-info-features__item')[2].text.strip())
                profile['Client focus'].append(soup.find_all(class_='company-info-features__item')[3].text.strip())
                profile['Headquarters'].append(soup.find_all(class_='company-info-features__item')[4].text.strip())
                profile['Founded'].append(soup.find_all(class_='company-info-features__item')[5].text.strip())
                if soup.find(class_='company-info__content') != None:
                    profile['description'].append(' '.join(soup.find(class_='company-info__content').text.split()))
                else:
                    profile['description'].append('')
                locations = ''
                for loc in range(len(soup.find_all(class_='country-and-city'))):
                    if loc != 1:
                        locations = locations + '\n' + soup.find_all(class_='country-and-city')[loc].text
                profile['Other locations'].append(locations.strip())
                linkedin = ''
                for link in soup.find_all(class_='company-locations__socials-link'):
                    if 'linkedin' in link['href']:
                        linkedin = link['href']
                        profile['linkedin'].append(linkedin)
                if len(linkedin) == 0:
                    profile['linkedin'].append(linkedin)
                clients= ''
                for client in range(len(soup.find_all(class_='major-client-item'))):
                    clients = clients + '\n' + soup.find_all(class_='major-client-item')[client].text
                profile['Major Clients'].append(clients.strip())
                headers = [item.text.strip() for item in soup.find_all(class_='category-block-header')]
                for ele in extra:
                    if ele in headers:
                        indx = headers.index(ele) 
                        rows = ''
                        for row in soup.find_all(class_='category-block-content')[indx].tbody.find_all('tr'):
                            row = row.find(class_='category-name').text.strip() + ' ' + row.find(class_='category-percentage').text.strip()
                            rows = rows + '\n' + row
                        profile[ele].append(rows.strip())
                    else:
                        profile[ele].append('')
                sleep(randint(3,10))
                df = pd.DataFrame(profile)
                with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
                    df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
                    df.to_excel(writer, sheet_name="result")
                print("company " + str(i+1) + ' scraped successfully and saved')
                driver.back()
            except Exception as e:
                print('company ' + str(i+1) + ' not able to load due to ' + str(e))
                driver.get(dataframe['link'][len(dataframe)-1]+str(page))
        count = count +1
    except Exception as e:
        print('page ' + str(page) + ' not able to load due to ' + str(e))
        driver.get(dataframe['link'][len(dataframe)-1]+str(page))
    
driver.quit()